***Половников Никита***

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp

# 1 Задание. 
## На платформе Kaggle или на Github найдите достаточно большой датасет с логами пользования сервисов

**Для работы был выбран датасет базы пользователей Netflix.**

https://www.kaggle.com/datasets/arnavsmayan/netflix-userbase-dataset/data

In [2]:
data = pd.read_csv('Netflix_Userbase.csv')

In [3]:
data

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month
...,...,...,...,...,...,...,...,...,...,...
2495,2496,Premium,14,25-07-22,12-07-23,Spain,28,Female,Smart TV,1 Month
2496,2497,Basic,15,04-08-22,14-07-23,Spain,33,Female,Smart TV,1 Month
2497,2498,Standard,12,09-08-22,15-07-23,United States,38,Male,Laptop,1 Month
2498,2499,Standard,13,12-08-22,12-07-23,Canada,48,Female,Tablet,1 Month


# 2 Задание
##  Разбейте на сегменты вашу аудиторию и примените для нескольких (примерно 3-4 метрики) метрик тесты Стьюдента, Манна-Уитни, Фишера (Фишера для нескольких сегментов)

### Метрики 
1. Средний Monthly Revenue
2. Возраст пользователей из США и Испании
3. Возраст пользователей с разными типами подписки (Basic, Premium, Standard)

***Зафиксируем для каждого теста уровень значимости*** 

$\alpha = 0.05$

### Проверка гипотезы равенства месячного дохода платформы двух группы пользователей. 
### (Тест Стьюдента)

In [4]:
data_1 = data[0:1250]
data_2 = data[1250:]

In [5]:
sp.ttest_ind(data_1['Monthly Revenue'], data_2['Monthly Revenue'])

Ttest_indResult(statistic=0.9603541923181456, pvalue=0.3369699285866148)

***p-val >*** $\normalsize \alpha$ ***=> принимаем нулевую гипотезу*** $(\mathbb{E}(a) = \mathbb{E}(b)$

### Проверка гипотезы о том, что возраст пользователей из США и Испании имеют одинаковое распределение.
### Тест Мана-Уитни

In [6]:
data['Country'].value_counts()

United States     451
Spain             451
Canada            317
United Kingdom    183
Australia         183
Germany           183
France            183
Brazil            183
Mexico            183
Italy             183
Name: Country, dtype: int64

In [7]:
age_country = data[['Country', 'Age']]
usa_age = age_country[age_country['Country'] == 'United States']['Age']
spain_age = age_country[age_country['Country'] == 'Spain']['Age']


In [8]:
sp.mannwhitneyu(usa_age, spain_age)

MannwhitneyuResult(statistic=102588.5, pvalue=0.8203894986672744)

***p-val >*** $\normalsize \alpha $ ***=> принимаем нулевую гипотезу о равенстве распределений возрастов пользователей из США и Испании.***


### Проверка гипотезы о том, что средний возраст пользователей с разными типами подписки одинаков.
### One-way ANOVA

In [9]:
data['Subscription Type'].value_counts()

Basic       999
Standard    768
Premium     733
Name: Subscription Type, dtype: int64

In [10]:
subs = data[['Subscription Type', 'Age']]
basic_age = subs[subs['Subscription Type'] == 'Basic']['Age']
standard_age = subs[subs['Subscription Type'] == 'Standard']['Age']
premium_age = subs[subs['Subscription Type'] == 'Premium']['Age']


In [11]:
sp.f_oneway(basic_age, standard_age, premium_age)

F_onewayResult(statistic=0.9630556399924789, pvalue=0.38186642866603326)

***p-val >*** $\normalsize \alpha $ ***=> принимаем нулевую гипотезу о том, что средний возраст пользователей с разными типами подписки одинаков.***


# 3 Задание
## Для >=3 сегментов сделайте попарные сравнения через тест Стьюдента и тест Фишера. Проверьте соотносятся ли результаты

### Проверка гипотезы равенства месячного дохода платформы двух группы пользователей попарными сравнениями через тест Стьюдента и тест Фишера.

In [12]:
part_1 = data[:833]
part_2 = data[833:1667]
part_3 = data[1667:2500]

***Тест Стьюдента***

In [13]:
p_val_student = []

p_val_student.append(sp.ttest_ind(part_1['Monthly Revenue'], part_2['Monthly Revenue']).pvalue)
p_val_student.append(sp.ttest_ind(part_1['Monthly Revenue'], part_3['Monthly Revenue']).pvalue)
p_val_student.append(sp.ttest_ind(part_2['Monthly Revenue'], part_3['Monthly Revenue']).pvalue)

np.mean(p_val_student)

0.8546869476604385

***Тест Фишера***

In [14]:
f_test_3_parts = sp.f_oneway(part_1['Monthly Revenue'], part_2['Monthly Revenue'], part_3['Monthly Revenue']).pvalue
f_test_3_parts

0.9614598333875473

### Как можно заметить, попарное сравнение t-критерием увеличивает ошибку первого рода

**Вероятность ошибки первого рода:**

In [15]:
1 - np.mean(p_val_student)/f_test_3_parts

0.11105288231429478

***Практическое значение 11% вполнее соотносится с табличным значением в 12% при количесве попарных сравнений равном 3.*** 

# 4 Задание
## Постройте точный и эфронов доверительные интервал для выбранных метрик. Проверьте соответствуют ли эти доверительные интервалы результатам теста Стьюдента.

### Метрики 
1. Средний Monthly Revenue
2. Возраст пользователей из США и Испании
3. Возраст пользователей с разными типами подписки (Basic, Premium, Standard)

### Построим точные доверительные интервалы.

***Точный доверительный интервал имеет вид:***
    $$ \left[ \overline x \pm t_{1-\frac{\alpha}{2}} \cdot \sqrt{\frac{\widehat s ^2}{n}} \right]$$

In [16]:
alpha = 0.05 

t_rv = sp.t(2500/2-1)
t_crit = t_rv.ppf(1-alpha/2)

left_1 = data_1['Monthly Revenue'].mean()- t_crit*data_1['Monthly Revenue'].std()/np.sqrt(2500/2)
right_1 = data_1['Monthly Revenue'].mean() + t_crit*data_1['Monthly Revenue'].std()/np.sqrt(2500/2)

print('Д.И. для группы 1: [', left_1, ',', right_1,']')

t_rv = sp.t(2500/2-1)
t_crit = t_rv.ppf(1-alpha/2)

left_2 = data_2['Monthly Revenue'].mean()- t_crit*data_2['Monthly Revenue'].std()/np.sqrt(2500/2)
right_2 = data_2['Monthly Revenue'].mean() + t_crit*data_2['Monthly Revenue'].std()/np.sqrt(2500/2)

print('Д.И. для группы 2: [', left_2, ',', right_2,']')

Д.И. для группы 1: [ 12.445391202300739 , 12.636208797699263 ]
Д.И. для группы 2: [ 12.38423483106162 , 12.567765168938381 ]


**Границы точных доверительных интервалов соответствуют результату теста Стьюдента из п.2 (**$\mathbb{E}(a) = \mathbb{E}(b)$**)**

In [17]:
alpha = 0.05 

t_rv = sp.t(451-1)
t_crit = t_rv.ppf(1-alpha/2)

left_1 = usa_age.mean()- t_crit*usa_age.std()/np.sqrt(451)
right_1 = usa_age.mean() + t_crit*usa_age.std()/np.sqrt(451)

print('Д.И. для возраста пользователя из США: [', left_1, ',', right_1,']')

t_rv = sp.t(451-1)
t_crit = t_rv.ppf(1-alpha/2)

left_2 = spain_age.mean()- t_crit*spain_age.std()/np.sqrt(451)
right_2 = spain_age.mean() + t_crit*spain_age.std()/np.sqrt(451)

print('Д.И. для возраста пользователя из Испании: [', left_2, ',', right_2,']')

Д.И. для возраста пользователя из США: [ 38.25657469024443 , 39.597083846340944 ]
Д.И. для возраста пользователя из Испании: [ 38.11005624886905 , 39.49083066909103 ]


In [18]:
sp.ttest_ind(usa_age, spain_age)

Ttest_indResult(statistic=0.25813072537598564, pvalue=0.7963650340536712)

***Границы точных доверительных интервалов соответствуют результату теста Стьюдента.***

In [19]:
alpha = 0.05 

t_rv = sp.t(len(basic_age)-1)
t_crit = t_rv.ppf(1-alpha/2)

left_1 = basic_age.mean()- t_crit*basic_age.std()/np.sqrt(len(basic_age))
right_1 = basic_age.mean() + t_crit*basic_age.std()/np.sqrt(len(basic_age))

print('Д. И. для возраста пользователя с подпиской Basic: [', left_1, ',', right_1,']')


t_rv = sp.t(len(standard_age)-1)
t_crit = t_rv.ppf(1-alpha/2)

left_2 = standard_age.mean()- t_crit*standard_age.std()/np.sqrt(len(standard_age))
right_2 = standard_age.mean() + t_crit*standard_age.std()/np.sqrt(len(standard_age))

print('Д. И. для возраста пользователя с подпиской Standard: [', left_2, ',', right_2,']')


t_rv = sp.t(len(premium_age)-1)
t_crit = t_rv.ppf(1-alpha/2)

left_3 = premium_age.mean()- t_crit*premium_age.std()/np.sqrt(len(premium_age))
right_3 = premium_age.mean() + t_crit*premium_age.std()/np.sqrt(len(premium_age))

print('Д. И. для возраста пользователя с подпиской Premium: [', left_3, ',', right_3,']', '\n')
      
      
p_val_student = []

p_val_student.append(sp.ttest_ind(basic_age, standard_age).pvalue)
p_val_student.append(sp.ttest_ind(basic_age, premium_age).pvalue)
p_val_student.append(sp.ttest_ind(standard_age, premium_age).pvalue)

print('P-value попарных тестов Стьюдента: ', p_val_student)
print('Среднее P-value попарных тестов Стьюдента = ', np.mean(p_val_student))

Д. И. для возраста пользователя с подпиской Basic: [ 38.38239281381608 , 39.27526484384157 ]
Д. И. для возраста пользователя с подпиской Standard: [ 38.5248506645779 , 39.51942016875543 ]
Д. И. для возраста пользователя с подпиской Premium: [ 37.98344830329315 , 39.042472569831 ] 

P-value попарных тестов Стьюдента:  [0.5714894850604375, 0.369678511422302, 0.16861968524485238]
Среднее P-value попарных тестов Стьюдента =  0.3699292272425307


**Границы точных доверительных интервалов соответствуют результату попарных тестов Стьюдента.**

## Построим эфроновы доверительные интервалы.

In [20]:
means_list_1 = []
means_list_2 = []
for _ in range(1000):
    temp_1 = np.random.choice(data_1['Monthly Revenue'], size = len(data_1), replace=True)
    means_list_1.append(np.mean(temp_1))
    temp_2 = np.random.choice(data_2['Monthly Revenue'], size = len(data_2), replace=True)
    means_list_2.append(np.mean(temp_2))


left_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[0]
right_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[1]

left_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[0]
right_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[1]

print('Д.И. для группы 1: [', left_1, ',', right_1,']')
print('Д.И. для группы 2: [', left_2, ',', right_2,']')


Д.И. для группы 1: [ 12.452 , 12.641 ]
Д.И. для группы 2: [ 12.388 , 12.568 ]


**Границы доверительных интервалов соответствуют результату теста Стьюдента из п.2 (**$\mathbb{E}(a) = \mathbb{E}(b)$**) и совпадают с границами точных доверительных интервалов**

In [21]:
means_list_1 = []
means_list_2 = []

for _ in range(1000):
    temp_1 = np.random.choice(usa_age, size = len(usa_age), replace=True)
    means_list_1.append(np.mean(temp_1))
    temp_2 = np.random.choice(spain_age, size = len(spain_age), replace=True)
    means_list_2.append(np.mean(temp_2))


left_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[0]
right_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[1]

left_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[0]
right_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[1]

print('Д.И. для возраста пользователя из США: [', left_1, ',', right_1,']')
print('Д.И. для возраста пользователя из Испании: [', left_2, ',', right_2,']')


Д.И. для возраста пользователя из США: [ 38.237 , 39.543 ]
Д.И. для возраста пользователя из Испании: [ 38.146 , 39.463 ]


**Границы доверительных интервалов соответствуют результату теста Стьюдента и почти равны границам точных доверительных интервалов**

In [22]:
means_list_1 = []
means_list_2 = []
means_list_3 = []

for _ in range(20000):
    temp_1 = np.random.choice(basic_age, size = len(basic_age), replace=True)
    means_list_1.append(np.mean(temp_1))
    temp_2 = np.random.choice(standard_age, size = len(standard_age), replace=True)
    means_list_2.append(np.mean(temp_2))
    temp_3 = np.random.choice(premium_age, size = len(premium_age), replace=True)
    means_list_3.append(np.mean(temp_3))

left_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[0]
right_1 = np.percentile(means_list_1, q=[2.5, 97.5]).round(3)[1]

left_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[0]
right_2 = np.percentile(means_list_2, q=[2.5, 97.5]).round(3)[1]

left_3 = np.percentile(means_list_3, q=[2.5, 97.5]).round(3)[0]
right_3 = np.percentile(means_list_3, q=[2.5, 97.5]).round(3)[1]

print('Д. И. для возраста пользователя с подпиской Basic: [', left_1, ',', right_1,']')
print('Д. И. для возраста пользователя с подпиской Standard: [', left_2, ',', right_2,']')
print('Д. И. для возраста пользователя с подпиской Premium: [', left_3, ',', right_3,']', '\n')

print('P-value попарных тестов Стьюдента', p_val_student)
print('Среднее P-value попарных тестов Стьюдента', np.mean(p_val_student))

Д. И. для возраста пользователя с подпиской Basic: [ 38.383 , 39.273 ]
Д. И. для возраста пользователя с подпиской Standard: [ 38.521 , 39.513 ]
Д. И. для возраста пользователя с подпиской Premium: [ 37.982 , 39.042 ] 

P-value попарных тестов Стьюдента [0.5714894850604375, 0.369678511422302, 0.16861968524485238]
Среднее P-value попарных тестов Стьюдента 0.3699292272425307


**Границы точных доверительных интервалов соответствуют результату попарных тестов Стьюдента.**

# 5 Задание
## Подумайте, можно ли использовать в пункте 2 другие стат. тесты. Приведите пару примеров.

**Возраст пользователей из США и Испании можно оценить с помощью теста Краскала.**

In [23]:
sp.kruskal(usa_age, spain_age)

KruskalResult(statistic=0.051607095023202744, pvalue=0.8202900379059674)

***p-val >*** $\normalsize \alpha $ ***=> принимаем нулевую гипотезу о равенстве распределений возрастов пользователей из США и Испании.***


**Проверку гипотезы равенства дисперсий месячного дохода платформы двух групп пользователей можно осуществить с помощью критерия Левена.**

In [24]:
sp.levene(data_1['Monthly Revenue'], data_2['Monthly Revenue'])

LeveneResult(statistic=2.7355544816893658, pvalue=0.09826307527318819)

***p-val >*** $\normalsize \alpha $ ***=> принимаем нулевую гипотезу о равенстве дисперсий месячного дохода платформы двух групп.***

**Результат теста Левена можно использовать для уточнения результата t-теста** (equal_var = True).